# Filtrado de **mediana** y tamaño de ventana

Este cuaderno ilustra cómo el **tamaño de la ventana** en un **filtro de mediana** afecta el resultado al eliminar **ruido sal y pimienta**. 

La analogía con el **tamaño de muestra** es la siguiente: ventanas muy pequeñas 
⇢ resultados inestables; ventanas adecuadas ⇢ buen equilibrio; ventanas demasiado grandes ⇢ sesgo por pérdida de detalle.

## Dependencias
Ejecuta esta celda una vez si estás en un entorno nuevo. Si ya instalaste, puedes omitirla.

In [ ]:
import sys, subprocess
pkgs = ['numpy', 'matplotlib', 'scikit-image', 'scipy']
for p in pkgs:
    try:
        __import__(p if p!='scikit-image' else 'skimage')
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', p])
print('Ready!')

## Cargar imagen y añadir ruido sal y pimienta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage.filters import median
from skimage.morphology import square
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim

image = img_as_float(data.camera())
noisy = random_noise(image, mode='s&p', amount=0.1)

# Mostrar imagen ruidosa (figura independiente, sin especificar colores ni estilos)
plt.figure(figsize=(5,5))
plt.imshow(noisy, cmap='gray')
plt.title('Imagen con ruido sal y pimienta (10%)')
plt.axis('off')
plt.show()

## Aplicar filtro de **mediana** con distintos tamaños de ventana
Probamos con ventanas cuadradas de 3×3, 7×7 y 15×15.

In [ ]:
window_sizes = [3, 7, 15]
results = {}
for w in window_sizes:
    filtered = median(noisy, footprint=square(w))
    results[w] = filtered
    # Figura independiente por resultado
    plt.figure(figsize=(5,5))
    plt.imshow(filtered, cmap='gray')
    plt.title(f'Filtro de mediana {w}x{w}')
    plt.axis('off')
    plt.show()

print('Hecho.')

## Métricas objetivas (PSNR y SSIM)
Calculamos PSNR y SSIM respecto a la imagen original para cada tamaño de ventana.

In [ ]:
import pandas as pd
rows = []
for w, im in results.items():
    v_psnr = psnr(image, im, data_range=1.0)
    v_ssim = ssim(image, im, data_range=1.0)
    rows.append({'Ventana': f'{w}x{w}', 'PSNR': v_psnr, 'SSIM': v_ssim})
df = pd.DataFrame(rows).sort_values('Ventana')
df

## Curvas PSNR/SSIM vs tamaño de ventana
Se grafican en figuras separadas (sin subplots).

In [ ]:
x = [int(v.split('x')[0]) for v in df['Ventana']]
plt.figure(figsize=(6,4))
plt.plot(x, df['PSNR'], marker='o')
plt.xlabel('Tamaño de ventana (lado)')
plt.ylabel('PSNR')
plt.title('PSNR vs tamaño de ventana (mediana)')
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(x, df['SSIM'], marker='o')
plt.xlabel('Tamaño de ventana (lado)')
plt.ylabel('SSIM')
plt.title('SSIM vs tamaño de ventana (mediana)')
plt.grid(True)
plt.show()

## Interpretación
- **Ventanas pequeñas (p. ej. 3×3):** eliminan parte del ruido pero pueden dejar granulado.
- **Ventanas intermedias (p. ej. 7×7):** buen equilibrio entre reducción de ruido y preservación de bordes.
- **Ventanas grandes (p. ej. 15×15):** remueven más ruido, pero **suavizan** y **pierden detalle**, similar a usar una *muestra excesivamente grande* en un estudio: la estimación se vuelve sesgada al promediar demasiado contexto y borrar matices.

En práctica, el tamaño óptimo se elige probando en un conjunto de validación o con criterios cuantitativos (PSNR/SSIM) según la *tarea*.